In [1]:
import os

In [2]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality sixth\\end-to-end-wine-quality-fifth\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality sixth\\end-to-end-wine-quality-fifth'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelPredictionConfig:
    model_path: Path
    preprocessor_path: Path
    params_path: Path
    train_array_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEME_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_prediction_config(self) -> ModelPredictionConfig:
        config = self.config.model_prediction

        model_prediction_config = ModelPredictionConfig(
            model_path= config.model_path,
            preprocessor_path=config.preprocessor_path,
            params_path=config.params_path,
            train_array_path=config.train_array_path
            
        )
        return model_prediction_config


[2024-01-27 12:40:17,330: INFO: utils: NumExpr defaulting to 4 threads.]


In [20]:
from ensure import ensure_annotations
import joblib
from pathlib import Path

def save_object(file_path, obj):
    try:
        dir_path = os.path.dirname(file_path)
        os.makedirs(dir_path, exist_ok=True)

        with open(file_path, 'wb') as file_obj:
            joblib.dump(obj, file_obj)

    except Exception as e:
        raise e
@ensure_annotations
def load_object(file_path: Path):
    try:
        with open(file_path, 'rb') as file_obj:
            return joblib.load(file_obj)
    except Exception as e:
        raise e

In [22]:
import pandas as pd

class ModelPrediction:
    def __init__(self, config: ModelPredictionConfig):
        self.config = config

    def get_data_as_data_frame(self, fixed_acidity: float, 
                 volatile_acidity: float, 
                 citric_acid: float, 
                 residual_sugar: float,
                 chlorides: float,
                 free_sulfur_dioxide: float,
                 total_sulfur_dioxide: float,
                 density: float,
                 pH: float,
                 sulphates: float,
                 alcohol: float):
        try:
            custom_data_input_dict = {
                "fixed_acidity": [fixed_acidity],
                "volatile_acidity": [volatile_acidity],
                "citric_acid": [citric_acid],
                "residual_sugar": [residual_sugar],
                "chlorides": [chlorides],
                "free_sulfur_dioxide": [free_sulfur_dioxide],
                "total_sulfur_dioxide": [total_sulfur_dioxide],
                "density": [density],
                "pH": [pH],
                "sulphates": [sulphates],
                "alcohol": [alcohol]
            }
            return pd.DataFrame(custom_data_input_dict)
        except Exception as e:
            raise e

    def predict_data(self, features):
        try:
            model_path = Path(self.config.model_path)
            preprocessor_path = Path(self.config.preprocessor_path)
            best_params_path = Path(self.config.params_path)
            train_array_path = Path(self.config.train_array_path)
            print("before loading")
            model = load_object(file_path= model_path)
            preprocessor = load_object(file_path=preprocessor_path)
            best_parameters = load_object(file_path=best_params_path)
            train_array = load_object(file_path= train_array_path)
            train_x, train_y = train_array[:,:-1], train_array[:,-1]
            model.set_params(**best_parameters)
            model.fit(train_x, train_y)
            print("After loading")
            data_scaled = preprocessor.transform(features)
            preds = model.predict(data_scaled)
            return preds
        except Exception as e:
            raise e
        
    

    

In [9]:
from flask import Flask , render_template, request
import os
import numpy as np
import pandas as pd


app = Flask(__name__)

@app.route('/', methods=['GET'])
def homePage():
    return render_template('index.html')

@app.route('/train', methods=['GET'])
def training():
    os.system('python main.py')
    return render_template('success.html')

@app.route('/predict', methods=['POST', 'GET'])
def predict():
    if request.method == "POST":
        try:
            data = CustomData(
            fixed_acidity = float(request.form.get("fixed_acidity")),
            volatile_acidity = float(request.form.get("volatile_acidity")),
            citric_acid = float(request.form.get("citric_acid")),
            residual_sugar = float(request.form.get("residual_sugar")),
            chlorides = float(request.form.get("chlorides")),
            free_sulfur_dioxide = float(request.form.get("free_sulfur_dioxide")),
            total_sulfur_dioxide = float(request.form.get("total_sulfur_dioxide")),
            density = float(request.form.get("density")),
            pH = float(request.form.get("pH")),
            sulphates = float(request.form.get("sulphates")),
            alcohol = float(request.form.get("alcohol"))
        )
            pred_df = data.get_data_as_data_frame()

            pred = predict_data(pred_df)
            


            return render_template('results.html', prediction = str(round(pred[0],3)))
        except Exception as e:
            print(f"The Exception message is : ",e)
            return render_template('error.html', error_message = e)


if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8080, debug=True)
    # app.run(host='0.0.0.0', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: on
[2024-01-26 16:40:10,723: INFO: _internal: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.188.20:8080]
[2024-01-26 16:40:10,725: INFO: _internal: Press CTRL+C to quit]
[2024-01-26 16:40:10,878: INFO: _internal:  * Restarting with watchdog (windowsapi)]


SystemExit: 1

c:\Users\User\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
